## LAB ØVELSE: Tid og tekst i Python

**Om øvelsen**

I denne øvelse skal anvendes teknikker inden for håndtering af tekstdata og datoer i Python. Øvelserne kan enten løses med basismoduler (fx `datetime`) eller via metoder i pandas. Der er flere måder at løse øvelserne på, men de vil kunne løses med de teknikker, som blev introduceret i de første tre Python sessioner i kurset.

**Formål**

Vi vil gerne undersøge udviklingen i, hvordan COVID-19 bliver diskuteret. Som case har vi udvalgt brancheforeningen Horesta (https://horesta.dk/). Som et første skridt i undersøgelsen, skal vi havde udvalgt de nyheder, som omhandler COVID-19.

**Om datasættet**

Datasættet består af alle nyheder fra Horesta fra april 2018 til starten af december 2020. Nyhederne er scrapet fra Horestas hjemmeside i starten af december 2020.

I finder datasættet i "data" mappen med navn "horesta_posts.json".

**1.**

Indlæs datasættet i Python. Overvej hvilken datastruktur som datasættet passer ind i. (se evt. `json.load()` funktionen fra pakken `json`: https://docs.python.org/3.8/library/json.html#json.load).

**2.**

Definér passende tekstsøgnigsstrenge (evt. med regular expressions), der kan bruges til at udvælge relevante tekster om COVID-19.

*Overvej*: COVID-19 nævnes ikke nødvendigvis eksplicit i alle tekster, der handler om COVID-19.

**3.**

Undersøg udviklingen i, hvor mange tekster, der handler om COVID-19 over tid.

*Bemærk*: Hver post i datasættet har en "publish_date" key. Denne indeholder datoen nyheden blev lagt op.

*Husk*: Grupperinger i data kan også foretages pba. tidsenheder.

## Løsningsforslag

In [ ]:
# indlæs pakker og sæt stier
import pandas as pd
from os.path import join
from datetime import datetime
import json
import re

data_dir = "/work/teaching-materials/data/horesta/horesta-posts_2018-2021.json" # absolut sti
#data_dir = join("..", "teaching-materials", "data", "horesta", "horesta-posts_2018-2021.json")  # relativ sti

### 1 - indlæs data

In [ ]:
# indlæs data

with open(data_dir, 'r') as f:
    posts = json.load(f)

In [ ]:
# hvad er det for noget? JSON records
posts[0]

In [ ]:
type(posts) # datasættet er en liste

In [ ]:
type(posts[0]) # hver indlæg i datasættet er en dictionary

In [ ]:
len(posts) # 1351 indlæg

#### Sidenote - skriv til fil i python

- "open" bruges til at skabe en filforbindelse
- 'r' bruges til at læse/åbne
- 'w' bruges til at skrive
- nedenstående danner filen 'mytext.txt' og skriver "Hello" og "Is this on?" ind i filen
- (der er sat betingelse rundt om, så I ikke ender med at skrive til filen, bare fordi I kører celler igennem)

In [ ]:
# sådan skriver man til fil

write_to_file = False

if write_to_file:
    with open('mytext.txt', 'w') as f:
        f.write('Hello')
        f.write('Is this on?')

### 2 - Søgning efter COVID-19

In [ ]:
pattern = re.compile(r"covid|corona|lockdown|vaccin|pandemi", re.IGNORECASE) # regex mønster

print(pattern.search(posts[0].get('text'))) # hvad indeholder søgningen?

print(bool(pattern.search(posts[0].get('text')))) # tvunget om til boolean - True/False

In [ ]:
# filtrering af data baseret på regex

posts_filtered = [] # tom liste

for post in posts: # loop igennem opslag/data
    if pattern.search(post.get('text')): # hvis mønsteret er i teksten, tilføj til liste
        posts_filtered.append(post)

In [ ]:
len(posts_filtered) # 291 matcher mønsteret

### 2 (data frame alternativ)

In [ ]:
posts_df = pd.DataFrame.from_records(posts) # datasæt er i JSON records format - kan dannes om til dataframe med denne funktion
posts_df.head()

In [ ]:
posts_filter_df = posts_df[posts_df['text'].str.contains(pattern, regex = True)] # bruger regex som filter
posts_filter_df.shape # også her matcher 291 mønsteret

In [ ]:
posts_filter_df.head()

#### Sidenote - brug af `.get()` i dictionaries

Man kan tilgå nøgler på to måder i dictionaries:
1. Tilgå nøglen med `[key]`
2. Tilgå nøglen med `.get(key)`

Forskellen er, at hvor 1 giver fejl, hvis nøglen ikke findes, så returnerer 2 None. 2 kan være en fordel, hvis man ved, at nøglen ikke findes i alle observationer i data. På den måde undgår man, at en kørsel afbrydes midt i det hele, fordi en nøgle mangler.

In [ ]:
posts[0]['ting'] # ting findes ikke - giver fejl

In [ ]:
posts[0].get('ting') # ting findes ikke - returnerer None

#### Sidenote - datoer i Python

Python skal fortælles, at noget er en dato. Læses blot som strings.

Datoer er tvetydige og findes i mange formater. Man konverterer string til dato ved at specificere dato formatet.

In [ ]:
datestring = "12/09 -22" # dato string - bemærk brug af /, - og mellemrum

date = datetime.strptime(datestring, "%d/%m -%y") # konverterer til dato med datetime.strptime - bemærk at /, - og mellemrum skal stå i formatet også

In [ ]:
date.month # tilgå deloplysning i dato - månedstal

### 3 - Udvikling over tid

Nedenstående fortæller python, at 'publish_date' feltet er en dato. Derefter laves felt for år-månedsværdier, så der kan optælles per måned.

Feltet for år-månedsværdier dannes ved at sætte år og måned sammen som en string; fx `'2019' + '02' = '201902'`

In [ ]:
# konverter datofelt til dato

for post in posts_filtered:  # loop igennem data
    parsed_date = datetime.strptime(post.get('publish_date'), '%d-%m - %Y') # omdanner til dato (format: dag-måned - år, fx "04-12 - 2020")
    
    post['publish_date'] = parsed_date # overskriver med formateret dato
    
    post['publish_yearmonth'] = str(parsed_date.year) + str(parsed_date.month).rjust(2, '0') # danner yearmonth oplysning (fx 201904). rjust sikrer, at string har karakterlængde på 2

In [ ]:
# tælling af nyheder over tid

# danner liste af unikke år-månedsværdier (set) og sorterer i rækkefølge
t_units = list(set([post.get('publish_yearmonth') for post in posts_filtered])) # set af tidsenheder 
t_units.sort() # sæt i rækkefølge

for yearmonth in t_units: # looper igennem hver tidsenhed
    
    t_collection = [post for post in posts_filtered if post.get('publish_yearmonth') == yearmonth] # samler indlæg fra pågældende år-månedsværdi i liste
    
    print(f'I {yearmonth} var der {len(t_collection)} nyheder på Horesta.dk omhandlende COVID-19') # danner print. antal opslag svarer til længden af listen
    

### 3 (data frame alternativ)

In [ ]:
# omdanner publish_date til datoformat med pd.to_datetime
# BEMÆRK: Koden giver en advarsel. Det er ikke en fejl. Lige denne advarsel (SettingWithCopyWarning) kan typisk ignoreres.

posts_filter_df['publish_date'] = pd.to_datetime(posts_filter_df['publish_date'], format = '%d-%m - %Y') # omdanner til dato 

In [ ]:
# eksmpel på at tilgå oplysinger i kolonne konverteret til datetime - .dt attributes
posts_filter_df['publish_date'].dt.month

In [ ]:
posts_filter_df['publish_date'].dt.year

In [ ]:
posts_filter_df['publish_date'].dt.quarter

In [ ]:
# nyeste dato svarer til højeste værdi
posts_filter_df['publish_date'].max()

In [ ]:
# danner grupperet dataframe, hvor data grupperes efter år og måned

posts_yearmonth_grouped = posts_filter_df.groupby([posts_filter_df['publish_date'].dt.year, posts_filter_df['publish_date'].dt.month]) # grupperer efter år-måned

In [ ]:
posts_yearmonth_grouped.size() # antal tekster i år-måned svarer til gruppestørrelsen 

In [ ]:
# kan visualiseres som barplot med .plot.bar()

posts_yearmonth_grouped.size().plot.bar() # ovenstående som søjlediagram